# Notebook 11: Advanced Models Part 3 (Trees & Random Forests)
Instead of using Linear Regression and its extensions, we could also use Random Forest Regression, where we can identify the most important predictors using feature importance, which is calculated based on the average reduction of the residual sum of squares (RSS) that results from splits using each feature. One important note is that we can only determine whether features are important or not; with this measure alone, we cannot conclude whether the important features lead to increases or decreases in the response.

In [1]:
# Import packages
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import statsmodels.formula.api as sm
import pickle
from collections import Counter
import sklearn as sk
from sklearn.cross_validation import LeaveOneOut
import StringIO
from sklearn.preprocessing import Imputer
from sklearn import linear_model
from sklearn import tree
from sklearn.ensemble import RandomForestRegressor as rf
%matplotlib inline
from sklearn.grid_search import GridSearchCV

/Users/philliphuang/anaconda/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/philliphuang/anaconda/lib/python2.7/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:
# Load cleaned NCD data from pickled files
out = open('data/clean/deaths_100k.p', 'r')
deaths_100k = pickle.load(out)
out.close()
out = open('data/clean/risk.p', 'r')
risk_of_death = pickle.load(out)
out.close()
out = open('data/clean/crops.p', 'r')
crops = pickle.load(out)
out.close()
out = open('data/clean/meat.p', 'r')
meat = pickle.load(out)
out.close()
out = open('data/clean/var_desc_livestock.p', 'r')
livestock_desc = pickle.load(out)
out.close()
out = open('data/clean/var_desc_crops.p', 'r')
crops_desc = pickle.load(out)
out.close()

# load cleaned food data with redundant variables removed
out = open('data/final/food_1970_2000_cleaned.p', 'r')
food_1970_2000_cleaned = pickle.load(out)

# Get response variables
deaths_100k_all_2000 = deaths_100k['all'][2000]
deaths_100k_cancer_2000 = deaths_100k['cancer'][2000]
deaths_100k_cardio_2000 = deaths_100k['cardio'][2000]
deaths_100k_diabetes_2000 = deaths_100k['diabetes'][2000]
deaths_100k_resp_2000 = deaths_100k['resp'][2000]
deaths_100k_all_2012 = deaths_100k['all'][2012]
deaths_100k_cancer_2012 = deaths_100k['cancer'][2012]
deaths_100k_cardio_2012 = deaths_100k['cardio'][2012]
deaths_100k_diabetes_2012 = deaths_100k['diabetes'][2012]
deaths_100k_resp_2012 = deaths_100k['resp'][2012]

risk_of_death_2000 = risk_of_death[2000]
risk_of_death_2012 = risk_of_death[2012]

time_period = range(1970, 2000)

# Calculate the mean for each crop/meat over the period 1970-2000
food_1970_2000 = pd.DataFrame(index=risk_of_death.index)

for crop in crops.iterkeys():
    food_1970_2000[crop] = crops[crop][time_period].mean(axis=1)
    
for m in meat.iterkeys():
    food_1970_2000[m] = meat[m][time_period].mean(axis=1)

# list of countries to drop due to being less than 50% full
countries_to_drop = []

for index, row in food_1970_2000.iterrows():
    if row.isnull().sum() > len(row) / 2:
        countries_to_drop.append(index)
        
# drop countries from response variables
deaths_100k_all_2000 = deaths_100k_all_2000.drop(countries_to_drop)
deaths_100k_cancer_2000 = deaths_100k_cancer_2000.drop(countries_to_drop)
deaths_100k_cardio_2000 = deaths_100k_cardio_2000.drop(countries_to_drop)
deaths_100k_diabetes_2000 = deaths_100k_diabetes_2000.drop(countries_to_drop)
deaths_100k_resp_2000 = deaths_100k_resp_2000.drop(countries_to_drop)

deaths_100k_all_2012 = deaths_100k_all_2012.drop(countries_to_drop)
deaths_100k_cancer_2012 = deaths_100k_cancer_2012.drop(countries_to_drop)
deaths_100k_cardio_2012 = deaths_100k_cardio_2012.drop(countries_to_drop)
deaths_100k_diabetes_2012 = deaths_100k_diabetes_2012.drop(countries_to_drop)
deaths_100k_resp_2012 = deaths_100k_resp_2012.drop(countries_to_drop)

risk_of_death_2000 = risk_of_death_2000.drop(countries_to_drop)
risk_of_death_2012 = risk_of_death_2012.drop(countries_to_drop)

In [3]:
# Plots bar graph of coefficient estimates for regression model and given significance level 
def plot_sig_coeffs(fit, sig_level):
    # Filter for predictors with p-values less than provided cutoff
    significant = fit.pvalues[fit.pvalues <= sig_level]
    # Plot coefficient estimates for predictors with p < 0.1 
    fit.params[significant.index].sort_values().plot(kind="bar", 
        title="Coefficient Estimates for Predictors with p < 0.1", 
        )

In [4]:
# a function to visualize trees
# Print decision tree model 'model', already fitted
def display_dt(model):
    dummy_io = StringIO.StringIO() 
    tree.export_graphviz(model, out_file = dummy_io) 
    print dummy_io.getvalue()

Like with Lasso, we can use GridSearchCV to identify the best combination of hyperparameters for the Random Forest, using Leave One Out as a cross validation method to account for overfitting. Note that we set the maximum depth of each tree in the forest to 5 and checking all the features before making each split, only tuning the number of trees in the forest. For the maximum depth, we wanted to limit the complexity of the trees because we care most about the features chosen for the earlier rather than later splits.

## Deaths per 100k from All Non-Communicable Diseases

In [5]:
# now to find the best hyperparameters for the random forest model
deaths_all_forest = rf(max_depth = 5, max_features = 'auto')
LOO = LeaveOneOut(n = deaths_100k_all_2000.shape[0])
params = {'n_estimators': range(20, 100, 10)}
deaths_all_forest_CV = GridSearchCV(deaths_all_forest, params, cv = LOO)
deaths_all_forest_CV.fit(food_1970_2000_cleaned, deaths_100k_all_2000)
print deaths_all_forest_CV.best_params_

{'n_estimators': 20}


Now we can use the identified best number of trees to learn the Random Forest model on the data and identify the 5 most important features from 100 iterations of the random forest model with the specified hyperparameters, then counting how many times each of these important features are selected by the different random forests.

In [6]:
important_features_deaths_all = []
for i in range(100):
    deaths_all_forest_best = rf(max_features = 'auto', max_depth = 5, n_estimators = 20)
    deaths_all_forest_best.fit(food_1970_2000_cleaned, deaths_100k_all_2000)
    # pulling the index of the features from the data
    features = list(food_1970_2000_cleaned.columns)
    # creating a dictionary with feature names as keys and feature importance values as values
    features_importance_deaths_all = dict(zip(features, deaths_all_forest_best.feature_importances_))
    # printing the most important features
    important_features_deaths_all.extend(sorted
                (features_importance_deaths_all, key = features_importance_deaths_all.get, reverse = True)[:5])
print Counter(important_features_deaths_all)

Counter({'Molluscs, Other': 100, 'Coffee and products': 97, 'Beer': 92, 'Oranges, Mandarines': 86, 'Cloves': 63, 'Palm Oil': 22, 'Oilcrops, Other': 15, 'Pineapples and products': 10, 'Beans': 5, 'Crustaceans': 3, 'Coconut Oil': 2, 'Wheat and products': 2, 'Bananas': 1, 'Maize and products': 1, 'Mutton & Goat Meat': 1})


According to these random forest models, Molluscs, Coffee, Beer, Oranges, and Cloves are among the most features for predicting age-standardized mortality rate for all non-communicable diseases, appearing in the 5 most important features for over half of the random forest models.

## Deaths per 100k from Cancer

In [7]:
# now to find the best hyperparameters for the random forest model
deaths_cancer_forest = rf(max_depth = 5, max_features = 'auto')
LOO = LeaveOneOut(n = deaths_100k_cancer_2000.shape[0])
params = {'n_estimators': range(20, 100, 10)}
deaths_cancer_forest_CV = GridSearchCV(deaths_cancer_forest, params, cv = LOO)
deaths_cancer_forest_CV.fit(food_1970_2000_cleaned, deaths_100k_cancer_2000)
print deaths_cancer_forest_CV.best_params_

{'n_estimators': 20}


In [8]:
important_features_deaths_cancer = []
for i in range(100):
    deaths_cancer_forest_best = rf(max_features = 'auto', max_depth = 5, n_estimators = 20)
    deaths_cancer_forest_best.fit(food_1970_2000_cleaned, deaths_100k_cancer_2000)
    # creating a dictionary with feature names as keys and feature importance values as values
    features_importance_deaths_cancer = dict(zip(features, deaths_cancer_forest_best.feature_importances_))
    # printing the most important features
    important_features_deaths_cancer.extend(sorted
                    (features_importance_deaths_cancer, key = features_importance_deaths_cancer.get, reverse = True)[:5])
print Counter(important_features_deaths_cancer)

Counter({'Potatoes and products': 100, 'Fruits, Other': 99, 'Apples and products': 73, 'Poultry Meat': 72, 'Beverages, Alcoholic': 49, 'Pulses, Other and products': 46, 'Mutton & Goat Meat': 17, 'Maize and products': 14, 'Honey': 10, 'Fats, Animals, Raw': 8, 'Pigmeat': 4, 'Palm Oil': 2, 'Nuts and products': 2, 'Meat, Other': 2, 'Butter, Ghee': 2})


According to these random forest models, Fruits, Potatoes, Poultry, Apples, and Pulses (such as Lentils, Chick Peas, and various Beans) are among the most important features for predicting age-standardized mortality rate for cancer, appearing in the 5 most important features for over half of the random forest models.

## Deaths per 100k from Cardiovascular Disease

In [9]:
# now to find the best hyperparameters for the random forest model
deaths_cardio_forest = rf(max_depth = 5, max_features = 'auto')
LOO = LeaveOneOut(n = deaths_100k_cardio_2000.shape[0])
params = {'n_estimators': range(20, 100, 10)}
deaths_cardio_forest_CV = GridSearchCV(deaths_cardio_forest, params, cv = LOO)
deaths_cardio_forest_CV.fit(food_1970_2000_cleaned, deaths_100k_cardio_2000)
print deaths_cardio_forest_CV.best_params_

{'n_estimators': 20}


In [10]:
important_features_deaths_cardio = []
for i in range(100):
    deaths_cardio_forest_best = rf(max_features = 'auto', max_depth = 5, n_estimators = 20)
    deaths_cardio_forest_best.fit(food_1970_2000_cleaned, deaths_100k_cardio_2000)
    # creating a dictionary with feature names as keys and feature importance values as values
    features_importance_deaths_cardio = dict(zip(features, deaths_cardio_forest_best.feature_importances_))
    # printing the most important features
    important_features_deaths_cardio.extend(sorted
                    (features_importance_deaths_cardio, key = features_importance_deaths_cardio.get, reverse = True)[:5])
print Counter(important_features_deaths_cardio)

Counter({'Wheat and products': 100, 'Molluscs, Other': 97, 'Palm Oil': 94, 'Oranges, Mandarines': 93, 'Pineapples and products': 50, 'Vegetables, Other': 14, 'Pulses, Other and products': 10, 'Maize and products': 7, 'Cloves': 6, 'Coffee and products': 5, 'Coconut Oil': 4, 'Groundnuts (Shelled Eq)': 3, 'Bananas': 2, 'Soyabean Oil': 2, 'Roots, Other': 2, 'Groundnut Oil': 2, 'Bovine Meat': 2, 'Beer': 2, 'Beans': 2, 'Crustaceans': 2, 'Oilcrops, Other': 1})


For age-standardized mortality rate for cardiovascular disease, Wheat, Molluscs, Oranges, and Palm Oil are among the most important features for prediction, appearing in the 5 most important features for over half of the random forest models.

## Deaths per 100k from Diabetes

In [11]:
# now to find the best hyperparameters for the random forest model
deaths_diabetes_forest = rf(max_depth = 5, max_features = 'auto')
LOO = LeaveOneOut(n = deaths_100k_diabetes_2000.shape[0])
params = {'n_estimators': range(20, 100, 10)}
deaths_diabetes_forest_CV = GridSearchCV(deaths_diabetes_forest, params, cv = LOO)
deaths_diabetes_forest_CV.fit(food_1970_2000_cleaned, deaths_100k_diabetes_2000)
print deaths_diabetes_forest_CV.best_params_

{'n_estimators': 20}


In [12]:
important_features_deaths_diabetes = []
for i in range(100):
    deaths_diabetes_forest_best = rf(max_features = 'auto', max_depth = 5, n_estimators = 20)
    deaths_diabetes_forest_best.fit(food_1970_2000_cleaned, deaths_100k_diabetes_2000)
    # creating a dictionary with feature names as keys and feature importance values as values
    features_importance_deaths_diabetes = dict(zip(features, deaths_diabetes_forest_best.feature_importances_))
    # printing the most important features
    important_features_deaths_diabetes.extend(sorted
                    (features_importance_deaths_diabetes, key = features_importance_deaths_diabetes.get, reverse = True)[:5])
print Counter(important_features_deaths_diabetes)

Counter({'Potatoes and products': 100, 'Sugar (Raw Equivalent)': 96, 'Apples and products': 83, 'Soyabean Oil': 78, 'Oilcrops Oil, Other': 42, 'Barley and products': 20, 'Spices, Other': 15, 'Palmkernel Oil': 13, 'Sugar, Raw Equivalent': 9, 'Pulses, Other and products': 9, 'Wine': 8, 'Cocoa Beans and products': 6, 'Groundnuts (Shelled Eq)': 5, 'Cottonseed Oil': 4, 'Wheat and products': 3, 'Cereals, Other': 2, 'Coconuts - Incl Copra': 2, 'Sesame seed': 1, 'Pigmeat': 1, 'Oats': 1, 'Freshwater Fish': 1, 'Coffee and products': 1})


For age-standardized mortality rate for diabetes, Potatoes, Sugar, Apples, and Soybean Oil are among the most important features for prediction, appearing in the 5 most important features for over half of the random forest models.

## Deaths per 100k from Respiratory Disease

In [13]:
# now to find the best hyperparameters for the random forest model
deaths_resp_forest = rf(max_depth = 5, max_features = 'auto')
LOO = LeaveOneOut(n = deaths_100k_resp_2000.shape[0])
params = {'n_estimators': range(20, 100, 10)}
deaths_resp_forest_CV = GridSearchCV(deaths_resp_forest, params, cv = LOO)
deaths_resp_forest_CV.fit(food_1970_2000_cleaned, deaths_100k_resp_2000)
print deaths_resp_forest_CV.best_params_

{'n_estimators': 20}


In [14]:
important_features_deaths_resp = []
for i in range(100):
    deaths_resp_forest_best = rf(max_features = 'auto', max_depth = 5, n_estimators = 20)
    deaths_resp_forest_best.fit(food_1970_2000_cleaned, deaths_100k_resp_2000)
    # creating a dictionary with feature names as keys and feature importance values as values
    features_importance_deaths_resp = dict(zip(features, deaths_resp_forest_best.feature_importances_))
    # printing the most important features
    important_features_deaths_resp.extend(sorted
                    (features_importance_deaths_resp, key = features_importance_deaths_resp.get, reverse = True)[:5])
print Counter(important_features_deaths_resp)

Counter({'Tomatoes and products': 96, 'Wine': 94, 'Pimento': 89, 'Poultry Meat': 66, 'Cocoa Beans and products': 49, 'Beer': 33, 'Coffee and products': 26, 'Beverages, Alcoholic': 14, 'Bovine Meat': 13, 'Spices, Other': 6, 'Offals, Edible': 4, 'Rice (Milled Equivalent)': 3, 'Fats, Animals, Raw': 2, 'Cream': 2, 'Potatoes and products': 1, 'Cassava and products': 1, 'Maize Germ Oil': 1})


For age-standardized mortality rate for respiratory disease, Wine, Pimento, Tomatoes, and Poultry are among the most important features for prediction, appearing in the 5 most important features for over half of the random forest models.

## Percentage Risk of Death from Non-Communicable Diseases
Lastly, we will also consider the random forests approach with the percentage risk of death from all non-communicable diseases as the response variable.

In [15]:
# now to find the best hyperparameters for the random forest model
risk_forest = rf(max_depth = 5, max_features = 'auto')
LOO = LeaveOneOut(risk_of_death_2000.shape[0])
params = {'n_estimators': range(20, 100, 10)}
risk_forest_CV = GridSearchCV(risk_forest, params, cv = LOO)
risk_forest_CV.fit(food_1970_2000_cleaned, risk_of_death_2000)
print risk_forest_CV.best_params_

{'n_estimators': 20}


In [16]:
important_features_risk = []
for i in range(100):
    risk_forest_best = rf(max_features = 'auto', max_depth = 5, n_estimators = 20)
    risk_forest_best.fit(food_1970_2000_cleaned, risk_of_death_2000)
    # creating a dictionary with feature names as keys and feature importance values as values
    features_importance_risk = dict(zip(features, risk_forest_best.feature_importances_))
    # printing the most important features
    important_features_risk.extend(sorted
                    (features_importance_risk, key = features_importance_risk.get, reverse = True)[:5])
print Counter(important_features_risk)

Counter({'Oranges, Mandarines': 100, 'Coffee and products': 100, 'Molluscs, Other': 99, 'Palm Oil': 40, 'Maize and products': 34, 'Oilcrops, Other': 33, 'Wheat and products': 21, 'Pulses, Other and products': 13, 'Butter, Ghee': 9, 'Cloves': 8, 'Potatoes and products': 8, 'Peas': 7, 'Beans': 7, 'Crustaceans': 6, 'Bovine Meat': 6, 'Palmkernel Oil': 4, 'Beer': 2, 'Apples and products': 1, 'Bananas': 1, 'Groundnuts (Shelled Eq)': 1})


For percentage risk of death from all non-communicable diseases, Molluscs, Oranges, and Coffee are among the most important features for prediction, appearing in the 5 most important features for over half of the random forest models.

In [17]:
# pickling the random forest models on the original data
pickle.dump(deaths_all_forest_best, open('data/models/deaths_all_forest_best.p', 'wb'))
pickle.dump(deaths_cancer_forest_best, open('data/models/deaths_cancer_forest_best.p', 'wb'))
pickle.dump(deaths_cardio_forest_best, open('data/models/deaths_cardio_forest_best.p', 'wb'))
pickle.dump(deaths_diabetes_forest_best, open('data/models/deaths_diabetes_forest_best.p', 'wb'))
pickle.dump(deaths_resp_forest_best, open('data/models/deaths_resp_forest_best.p', 'wb'))
pickle.dump(risk_forest_best, open('data/models/risk_forest_best.p', 'wb'))